In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
%matplotlib inline

In [3]:
data = pd.read_csv('words_values_data_raunaq.csv')
data=pd.concat([data[data['files']==x] for x in np.unique(data.files) if np.array(data[data['files']==x].labels).sum() > 0],axis=0).reset_index(drop=True)

In [4]:
data.head()

,words,coords,files,labels
0,TAX,"[1080, 63, 1143, 105]",12222018075543.jpg,0
1,INVOICE,"[1166, 62, 1323, 102]",12222018075543.jpg,0
2,GSTIN,"[94, 143, 208, 192]",12222018075543.jpg,0
3,03AUVPS8413J121,"[250, 140, 610, 184]",12222018075543.jpg,0
4,ORIGINAL,"[1891, 47, 2085, 94]",12222018075543.jpg,0


In [5]:
words=[data[data['files']==x].shape[0] for x in np.unique(data.files)]

In [6]:
np.array(words).mean(),np.array(words).std(),np.median(np.array(words)),np.array(words).min(),np.array(words).max()

(179.24858757062148, 58.58318062103969, 160.5, 84, 401)

In [7]:
np.where(np.array(words)<300)[0].shape

(337,)

In [8]:
sentence=np.array([(x,' '.join([str(y) for y in list(data[data['files']==x].words)]),''.join([str(y) for y in list(data[data['files']==x].labels)])) for x in np.unique(data.files)])

In [9]:
text = [x[1] for x in sentence]
files= [x[0] for x in sentence]
labels = [x[-1] for x in sentence]

In [10]:
df_sen = pd.DataFrame([files,text,labels]).T
df_sen.columns=['files','text','labels']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_sen.text,df_sen.labels, test_size=0.10, random_state=42)

In [28]:
x=[X_train, X_val, y_train, y_val]

In [12]:
import pickle
dbpath = open('train.pk','wb')
pickle.dump(x,dbpath)

In [1]:
import pickle

X_train, X_val, y_train, y_val=pickle.load(open('train.pk','rb'))

In [26]:
f_te[9],X_val[9]

('MOBILE BILL.jpg',
 'GSTIN 03AAQFB6165R1ZS TAX INVOICE (Original for Buyer)\n TIN 03722201169 Phone No: 98140-20241 98100-09439\n BPR ZIPPERS Manufacturers of: CFC ZIPPER ROLLS\n F-295, PHASE-VIII, FOCAL POINT, LUDHIANA-141010 (PUNJAB) INDIA. E-mail id bprzippers@gmail.com\n MIS EXCEL TRADING COMPANY Invoice 204 Dated 30/06/2018\n 9/952,ANANDPURI COLONY Order No: Dated :\n NOORWALA ROAD\n LUDHIANA\n State PUNJAB G.R. No: Dated: 11\n GSTIN 03AUWPS7596M1ZC PAN Transport:\n Ph. No: 9888886346 Email ID: Vehicle No: No.of Pack:\n P.mark No Weight: 31.000\n Sr. Description of goods HSN Qnty Per Rate Amount C-GST S-GST I-GST TOTAL\n No\n Code Rate Amoun Rate Amount Rate Amoun\n ZIPPER ROLL\n 96072000 30.000 350.00 10,500.00 9.00 945.00 9.000 945.00 12,390.00\n ZIPPER ROLL\n 96072000 1.000 400.00 400.00 9.00 36.00 9.000 36.00 472.00 Total 31.000 10,900.00 12,862.00 Remarks: 981.00| 981.00\n Total Taxable Value\n Total C-GST Amount\n Total S-GST Amount 10900.00 981.00\n 981.00 GRAND TOTAL 12,8

In [20]:
f_tr=list(np.unique(df_sen.loc[X_train.index]['files']))
f_te=list(np.unique(df_sen.loc[X_val.index]['files']))

In [18]:
o_test=['12222018090033.jpg',
 'rudramotors-sale-2-01.png',
 'invoices1-087.png',
 'purchasebillstvs-29.png',
 'Sale of TVS AGENCY 31.10.1825112018131725_010.jpg',
 'Sale of TVS AGENCY 31.10.1825112018131727_013.jpg',
 'SN_aggarwal_29Nov.jpg',
 'rudramotors-sale-1-09.png',
 '12222018081345.jpg',
 '18-Dec-Purchase-12.png',
 'Sale of TVS AGENCY 31.10.1825112018131739_033.jpg',
 'img34.jpg',
 '18-Dec-Exp-04.png',
 'img26.jpg',
 '18-Dec-Purchase-07.png',
 'PURCHASE OF NATIONAL TECH TRIAL05122018171245_006.jpg',
 '12222018081237.jpg',
 'rudramotors-sale-1-43.png',
 'purchasebillstvs-22.png',
 'img44.jpg',
 'gurjanwal_1Dec.jpg',
 'shade_16_nov.jpg',
 'img28.jpg',
 'invoices1-023.png',
 'invoices1-035.png',
 '12222018085524.jpg',
 'Sale of TVS AGENCY 31.10.1825112018131738_032.jpg',
 '18-Dec-Exp-05.png',
 'PURCHASE BILL.jpg',
 'rudramotors-sale-1-59.png',
 'PURCHASE OF NATIONAL TECH TRIAL05122018171243_001.jpg',
 'PURCHASE OF NATIONAL TECH TRIAL05122018171246_007.jpg',
 'purchasebills-23.png',
 'Purchase of TVS AGENCY 31.10.1825112018114910_005.jpg',
 'Purchase of TVS AGENCY 31.10.1825112018114908_001.jpg',
 'invoices1-031.png',
 'purchasebills-20.png',
 'Purchase of TVS AGENCY 31.10.1825112018114911_007.jpg']

In [29]:
[x for x in o_test if x not in f_tr]

['rudramotors-sale-1-09.png']

## BERT Token classification

In [2]:
from pathlib import Path
import torch
from torch import nn
# import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForTokenClassification

import pytorch_pretrained_bert as _bert

import numpy as np

def flatten(list_of_lists):
    for list in list_of_lists:
        for item in list:
            yield item



_device = torch.device("cuda")
class Bert():

    MASK = '[MASK]'
    CLS = "[CLS]"
    SEP = "[SEP]"

#     supported_langs = set(lines(
#         Path(__file__).parent / "data" / "bert_langs.wiki"))

    def __init__(self, model, model_name, device=None, half_precision=False):
        super().__init__()
        self.model_name = model_name
        self.device = device or _device
        do_lower_case = "uncased" in model_name
        self.tokenizer = _bert.BertTokenizer.from_pretrained(
            self.model_name, do_lower_case=do_lower_case)
        maybe_model_wrapper = model.from_pretrained(model_name).to(
            device=self.device)
        try:
            self.model = maybe_model_wrapper.bert
        except AttributeError:
            self.model = maybe_model_wrapper
        if half_precision:
            self.model.half()
        self.max_len = \
            self.model.embeddings.position_embeddings.weight.size(0)
        self.dim = self.model.embeddings.position_embeddings.weight.size(1)

    def tokenize(self, text, masked_idxs=None):
        tokenized_text = self.tokenizer.tokenize(text)
        if masked_idxs is not None:
            for idx in masked_idxs:
                tokenized_text[idx] = self.MASK
        # prepend [CLS] and append [SEP]
        # see https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_classifier.py#L195  # NOQA
        tokenized = [self.CLS] + tokenized_text + [self.SEP]
        return tokenized

    def tokenize_to_ids(self, text, masked_idxs=None, pad=True):
        tokens = self.tokenize(text, masked_idxs)
        return self.convert_tokens_to_ids(tokens, pad=pad)

    def convert_tokens_to_ids(self, tokens, pad=True):
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        ids = torch.tensor([token_ids]).to(device=self.device)
        assert ids.size(1) < self.max_len
        if pad:
            padded_ids = torch.zeros(1, self.max_len).to(ids)
            padded_ids[0, :ids.size(1)] = ids
            mask = torch.zeros(1, self.max_len).to(ids)
            mask[0, :ids.size(1)] = 1
            return padded_ids, mask
        else:
            return ids

    def subword_tokenize(self, tokens):
        """Segment each token into subwords while keeping track of
        token boundaries.
        Parameters
        ----------
        tokens: A sequence of strings, representing input tokens.
        Returns
        -------
        A tuple consisting of:
            - A list of subwords, flanked by the special symbols required
                by Bert (CLS and SEP).
            - An array of indices into the list of subwords, indicating
                that the corresponding subword is the start of a new
                token. For example, [1, 3, 4, 7] means that the subwords
                1, 3, 4, 7 are token starts, while all other subwords
                (0, 2, 5, 6, 8...) are in or at the end of tokens.
                This list allows selecting Bert hidden states that
                represent tokens, which is necessary in sequence
                labeling.
        """
        subwords = list(map(self.tokenizer.tokenize, tokens))
        subword_lengths = list(map(len, subwords))
        subwords = [self.CLS] + list(flatten(subwords)) + [self.SEP]
        token_start_idxs = 1 + np.cumsum([0] + subword_lengths[:-1])
        return subwords, token_start_idxs

    def subword_tokenize_to_ids(self, tokens):
        """Segment each token into subwords while keeping track of
        token boundaries and convert subwords into IDs.
        Parameters
        ----------
        tokens: A sequence of strings, representing input tokens.
        Returns
        -------
        A tuple consisting of:
            - A list of subword IDs, including IDs of the special
                symbols (CLS and SEP) required by Bert.
            - A mask indicating padding tokens.
            - An array of indices into the list of subwords. See
                doc of subword_tokenize.
        """
        subwords, token_start_idxs = self.subword_tokenize(tokens)
        subword_ids, mask = self.convert_tokens_to_ids(subwords)
        token_starts = torch.zeros(1, self.max_len).to(subword_ids)
        token_starts[0, token_start_idxs] = 1
        return subword_ids, mask, token_starts

    def segment_ids(self, segment1_len, segment2_len):
        ids = [0] * segment1_len + [1] * segment2_len
        return torch.tensor([ids]).to(device=self.device)

    @staticmethod
    def Model(model_name, **kwargs):
        return Bert(_bert.BertModel, model_name, **kwargs)

    @staticmethod
    def ForMaskedLM(model_name, **kwargs):
        return Bert(_bert.BertForMaskedLM, model_name, **kwargs)

    @staticmethod
    def ForSequenceClassification(model_name, **kwargs):
        return Bert(
            _bert.BertForSequenceClassification, model_name, **kwargs)

    @staticmethod
    def ForNextSentencePrediction(model_name, **kwargs):
        return Bert(_bert.BertForNextSentencePrediction, model_name, **kwargs)

    @staticmethod
    def ForForPreTraining(model_name, **kwargs):
        return Bert(_bert.BertForPreTraining, model_name, **kwargs)

    @staticmethod
    def ForForQuestionAnswering(model_name, **kwargs):
        return Bert(_bert.BertForQuestionAnswering, model_name, **kwargs)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
bert = Bert(BertModel,'bert-base-cased')


In [4]:
np.argmin(np.array([len(x.split(' ')) for x in X_train]))

41

In [5]:
len(list(X_train)[41].split(' '))

84

In [6]:
tokens=list(X_train)[41]
featurized_sentences = []

tokens=tokens.split(' ')

features = {}

features["bert_ids"], features["bert_mask"], features["bert_token_starts"] = bert.subword_tokenize_to_ids(tokens)
featurized_sentences.append(features)


In [18]:
featurized_sentences = []
for j,tokens in enumerate(X_train):
    tokens=tokens.split(' ')
    try:
        features = {}

        features["bert_ids"], features["bert_mask"], features["bert_token_starts"] = bert.subword_tokenize_to_ids(tokens)
        featurized_sentences.append(features)
    except:
        featurized_sentences

In [7]:
def collate_fn(featurized_sentences_batch):
    bert_batch = [torch.cat([features[key] for features in featurized_sentences], dim=0) for key in ("bert_ids", "bert_mask", "bert_token_starts")]
    return bert_batch

In [8]:
class SequenceTagger(torch.nn.Module):
     def __init__(self, data_parallel=True):
            bert = BertModel.from_pretrained("bert-base-cased").to(device=torch.device("cuda"))
            self.bert = bert
            bert_dim = 786 # (or get the dim from BertEmbeddings)
            n_labels = 5  # need to set this for your task
            self.out = torch.nn.Linear(bert_dim, n_labels)
            #...  # droput, log_softmax...
    
     def forward(self, bert_batch, true_labels):
            bert_ids, bert_mask, bert_token_starts = bert_batch
            # truncate to longest sequence length in batch (usually much smaller than 512) to save GPU RAM
            max_length = (bert_mask != 0).max(0)[0].nonzero()[-1].item()
            if max_length < bert_ids.shape[1]:
                  bert_ids = bert_ids[:, :max_length]
                  bert_mask = bert_mask[:, :max_length]

            segment_ids = torch.zeros_like(bert_mask)  # dummy segment IDs, since we only have one sentence
            bert_last_layer = self.bert(bert_ids, segment_ids)[0][-1]
            # select the states representing each token start, for each instance in the batch
            bert_token_reprs = [
                   layer[starts.nonzero().squeeze(1)]
                   for layer, starts in zip(bert_last_layer, bert_token_starts)]
            # need to pad because sentence length varies
            padded_bert_token_reprs = pad_sequence(
                   bert_token_reprs, batch_first=True, padding_value=-1)
            # output/classification layer: input bert states and get log probabilities for cross entropy loss
            pred_logits = self.log_softmax(self.out(self.dropout(padded_bert_token_reprs)))
            mask = true_labels != -1  # I did set label = -1 for all padding tokens somewhere else
            loss = cross_entropy(pred_logits, true_labels)
            # average/reduce the loss according to the actual number of of predictions (i.e. one prediction per token).
            loss /= mask.float().sum()
            return loss

In [9]:
# collate
bert_batch=collate_fn(featurized_sentences[0])

In [10]:
bert_ids, bert_mask, bert_token_starts = bert_batch

In [15]:
bert_batch.size()

AttributeError: 'list' object has no attribute 'size'

In [11]:
max_length = (bert_mask != 0).max(0)[0].nonzero()[-1].item()

In [12]:
max_length,bert_ids.shape[1]

(227, 512)

In [64]:
if max_length < bert_ids.shape[1]:
                  bert_ids = bert_ids[:, :max_length]
                  bert_mask = bert_mask[:, :max_length]

In [65]:
segment_ids = torch.zeros_like(bert_mask)

In [66]:
model=BertModel.from_pretrained("bert-base-cased").to(device=torch.device("cuda"))

In [67]:
bert_last_layer = model(bert_ids, segment_ids)[0][-1]

In [68]:
bert_last_layer.size()

torch.Size([1, 416, 768])

In [69]:
bert_token_reprs = [
                   layer[starts.nonzero().squeeze(1)]
                   for layer, starts in zip(bert_last_layer, bert_token_starts)]

In [70]:
from torch.nn.utils.rnn import pad_sequence
padded_bert_token_reprs = pad_sequence(
                   bert_token_reprs, batch_first=True, padding_value=-1)

In [71]:
out = torch.nn.Linear(768, 5).cuda()

In [72]:
import torch.nn.functional as F
true_labels=torch.tensor([int(x) for x in list(list(y_train)[40])]).long()
cross_entropy = torch.nn.CrossEntropyLoss()


In [78]:
# output/classification layer: input bert states and get log probabilities for cross entropy loss
pred_logits = F.log_softmax(out(padded_bert_token_reprs))
mask = true_labels != -1  # I did set label = -1 for all padding tokens somewhere else
loss = cross_entropy(pred_logits.cpu().view([pred_logits.cpu().size()[1],5]), true_labels)
# average/reduce the loss according to the actual number of of predictions (i.e. one prediction per token).
# loss /= mask.float().sum()

/home/raghu/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [41]:
padded_bert_token_reprs.size()

torch.Size([1, 84, 768])

In [52]:
pred_logits.cpu().view([84,5]).size()

torch.Size([84, 5])

In [77]:
pred_logits.cpu().size()[1]

129

In [55]:
mask.float().sum()

tensor(84.)

In [80]:
loss.backward()

In [81]:
loss

tensor(1.6094, grad_fn=<NllLossBackward>)